In [2]:
!pip install transformers datasets



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


In [4]:

# Load dataset
dataset = load_dataset("glue", "mrpc")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [5]:

# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True,padding="max_length", max_length=128)


In [7]:

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [8]:

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)


In [9]:

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)


In [15]:
# Evaluate before training
print("Evaluation before training:")
metrics_before = trainer.evaluate()
print(metrics_before)

Evaluation before training:


{'eval_loss': 0.741563081741333, 'eval_runtime': 160.7936, 'eval_samples_per_second': 2.537, 'eval_steps_per_second': 0.162}


In [10]:

# Train model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.452931
2,No log,0.389281
3,0.448600,0.442915


TrainOutput(global_step=690, training_loss=0.38620324618574503, metrics={'train_runtime': 256.2354, 'train_samples_per_second': 42.945, 'train_steps_per_second': 2.693, 'total_flos': 723818513295360.0, 'train_loss': 0.38620324618574503, 'epoch': 3.0})

In [12]:
# Evaluate after training
print("Evaluation after training:")
metrics_after = trainer.evaluate()
print(metrics_after)

Evaluation after training:


{'eval_loss': 0.4429146945476532, 'eval_runtime': 2.7357, 'eval_samples_per_second': 149.138, 'eval_steps_per_second': 9.504, 'epoch': 3.0}


In [17]:
def test_sentence_pair(sentence1, sentence2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(sentence1, sentence2, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to the same device
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    print(f"Sentence 1: {sentence1}")
    print(f"Sentence 2: {sentence2}")
    print(f"Prediction: {'Paraphrase' if predictions == 1 else 'Not a paraphrase'}")


In [19]:

# Test with some custom examples
test_sentence_pair("The cat is on the mat.", "A cat is sitting on the mat.")
test_sentence_pair("The weather is nice today.", "I enjoy reading books.")

Sentence 1: The cat is on the mat.
Sentence 2: A cat is sitting on the mat.
Prediction: Paraphrase
Sentence 1: The weather is nice today.
Sentence 2: I enjoy reading books.
Prediction: Not a paraphrase
